Step 1: Load Dataset

Pada langkah pertama ini, kita akan mengimpor library pandas dan membaca dataset hasil scraping review dari Steam. Dataset ini berisi ulasan game yang nantinya akan kita olah lebih lanjut untuk analisis sentimen.

Metode yang digunakan:

    pd.read_csv() digunakan untuk membaca file .csv.

    df.head() digunakan untuk menampilkan 5 baris pertama dari dataset agar kita bisa melihat struktur datanya.

In [29]:
import pandas as pd
import string
import re
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.utils import to_categorical


In [30]:
# Baca file hasil scraping
df = pd.read_csv('https://raw.githubusercontent.com/jurus231/PROJ.Sentiment-Analisis/refs/heads/main/steam_reviews_labeled.csv')

# Tampilkan 5 data pertama
df.head()

,review,sentiment
0,Enjoyable and calm game.,positive
1,very good game,positive
2,"In it's current state, I wouldn't recommend th...",positive
3,mantap\r\n,neutral
4,"bugged mess, worst than lol VALVe pls fix",negative


Step 2: Preprocessing Teks

Langkah ini bertujuan untuk membersihkan data teks agar siap diproses oleh model machine learning. Pembersihan ini dilakukan melalui beberapa tahapan penting:

    Lowercase: Mengubah semua huruf menjadi huruf kecil agar konsisten.

    Hapus URL: Menghapus tautan yang tidak relevan dengan analisis sentimen.

    Hapus angka dan tanda baca: Untuk menyederhanakan teks.

    Stopwords removal: Menghapus kata-kata umum seperti "the", "and", "is", dll., yang tidak memiliki makna penting dalam analisis sentimen.

Setelah diproses, hasil teks bersih disimpan dalam kolom baru bernama clean_review.

In [31]:
# Jalankan ini sekali untuk download stopwords
import nltk
nltk.download('stopwords')

# Definisikan stopwords bahasa Inggris
stop_words = set(stopwords.words('english'))

# Fungsi preprocessing
def preprocess_text(text):
    # 1. Ubah ke huruf kecil
    text = text.lower()
    # 2. Hapus URL
    text = re.sub(r'http\S+', '', text)
    # 3. Hapus angka dan tanda baca
    text = text.translate(str.maketrans('', '', string.punctuation + string.digits))
    # 4. Tokenisasi dan hapus stopwords
    tokens = text.split()
    tokens = [word for word in tokens if word not in stop_words]
    return ' '.join(tokens)

# Terapkan preprocessing ke kolom 'review'
df['clean_review'] = df['review'].astype(str).apply(preprocess_text)

# Tampilkan hasil
df[['review', 'clean_review']].head()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\GEMINK1\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,review,clean_review
0,Enjoyable and calm game.,enjoyable calm game
1,very good game,good game
2,"In it's current state, I wouldn't recommend th...",current state wouldnt recommend game major cha...
3,mantap\r\n,mantap
4,"bugged mess, worst than lol VALVe pls fix",bugged mess worst lol valve pls fix


Step 3: Ekstraksi Fitur dan Pembagian Dataset

Agar data teks bisa diproses oleh algoritma machine learning, kita perlu mengubahnya menjadi representasi numerik. Di langkah ini, kita menggunakan TF-IDF (Term Frequency-Inverse Document Frequency) untuk mengubah kata menjadi angka.

    TfidfVectorizer(max_features=5000) menghasilkan representasi numerik berdasarkan kata-kata yang paling informatif di seluruh dataset.

    Dataset kemudian dibagi menjadi training set dan testing set dengan rasio 80:20 menggunakan train_test_split.

In [32]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

# Ekstrak fitur dari kolom clean_review
vectorizer = TfidfVectorizer(max_features=5000)
X = vectorizer.fit_transform(df['clean_review'])

# Label target
y = df['sentiment']

# Split data: 80% train, 20% test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print(f'Ukuran data latih: {X_train.shape[0]}')
print(f'Ukuran data uji: {X_test.shape[0]}')


Ukuran data latih: 2993
Ukuran data uji: 749


Step 4: Training dan Evaluasi Model

Pada tahap ini, kita melatih model klasifikasi menggunakan Logistic Regression, salah satu algoritma yang sering digunakan untuk analisis sentimen.

    model.fit() digunakan untuk melatih model pada data latih.

    model.predict() digunakan untuk memprediksi sentimen dari data uji.

    accuracy_score dan classification_report digunakan untuk mengevaluasi performa model.

In [33]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

# Inisialisasi dan training model
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

# Prediksi
y_pred = model.predict(X_test)

# Evaluasi
acc = accuracy_score(y_test, y_pred)
print(f"Akurasi: {acc * 100:.2f}%\n")
print("Classification Report:")
print(classification_report(y_test, y_pred))


Akurasi: 95.33%

Classification Report:
              precision    recall  f1-score   support

    negative       1.00      0.29      0.44         7
     neutral       0.84      0.98      0.91       175
    positive       0.99      0.95      0.97       567

    accuracy                           0.95       749
   macro avg       0.95      0.74      0.78       749
weighted avg       0.96      0.95      0.95       749



In [34]:
import joblib

# Simpan model
joblib.dump(model, 'sentiment_logreg_model.pkl')

# Simpan vectorizer
joblib.dump(vectorizer, 'tfidf_vectorizer.pkl')

print("Model dan vectorizer berhasil disimpan!")


Model dan vectorizer berhasil disimpan!
